两个抽象:

- 流
- 路径名(pathname)

# 读取文件数据

In [ ]:
; OPEN: 默认返回基于字符的输入流
; READ-CHAR: 读取单个字符
; READ-LINE: 读取一行文本
; READ: 读取一个S-表达式, 并返回一个Lisp对象
; CLOSE: 关闭流

In [1]:
; 读取文件第一行
(let ((in (open "data/name.txt")))
     (format t "~a~%" (read-line in))
     (close in))

T

Alice


In [2]:
; :if-does-not-exist: 指定OPEN的行为, 可选值:
; :error: 报错(默认)
; :create: 创建该文件
; NIL: 返回NIL来代替流

In [4]:
; READ-CHAR, READ-LINE, READ的可选参数: eof-error-p
; 默认为真, 指定当函数在文件结尾处被调用时是否报错
; 如果eof-error-p为false, 返回eof-value(默认为NIL)

(let ((in (open "data/name.txt" :if-does-not-exist nil)))
     (when in
         (loop for line = (read-line in nil)
               while line do (format t "~a~%" line)))
     (close in))

T

Alice
Bob
Cartman
David
Eve
Frank
Green


In [5]:
; READ: 读取S-表达式
; see also PRINT
(defparameter *s* (open "data/s-expression.txt"))

*S*

In [6]:
(read *s*)

(1 2 3)

In [7]:
(read *s*)

456

In [8]:
(read *s*)

"a string"

In [9]:
(read *s*)

((A B) (C D))

In [10]:
(close *s*)

T

# 读取二进制数据

In [ ]:
; :element-type '(unsigned-byte 8)
; READ-BYTE

In [21]:
(let ((bf (open "data/binary-output" 
                :element-type '(unsigned-byte 8))))
     (read-byte bf)
     (read-byte bf)
     (close bf))

T

# 批量读取

In [25]:
; READ-SEQUENCE
; 

(defvar *data* (make-array 15 :initial-element nil))
(values (read-sequence *data* (make-string-input-stream "test string")) *data*)

*DATA*

11

#(#\t #\e #\s #\t #\  #\s #\t #\r #\i #\n #\g NIL NIL NIL NIL)

# 文件输出

In [ ]:
; direction :output
; if-exists :supersede/:append/:overwrite/NIL

; WRITE-CHAR
; WRITE-LINE
; WRITE-STRING
; TERPRI
; FRESH-LINE
;
; PRINT
; PRIN1
; PPRINT
; PRINC

; WRITE-BYTE
; WRITE-SEQUENCE

In [14]:

(let ((bf (open "data/binary-output" 
                :direction :output 
                :if-exists :supersede
                :element-type '(unsigned-byte 8))))
     (write-byte 100 bf)
     (write-byte 200 bf)
     (close bf))

T

# 关闭文件

In [ ]:
; WITH-OPEN-FILE宏: 构建在UNWIND-PROTECT特殊操作符

In [22]:
(with-open-file (is "data/name.txt")
    (format t "~a~%" (read-line is)))

NIL

Alice


In [24]:
(with-open-file (os "data/binary-output"
                    :direction :output 
                    :if-exists :supersede
                    :element-type '(unsigned-byte 8))
    (write-byte 101 os)
    (write-byte 201 os))

(with-open-file (in "data/binary-output"
                    :element-type '(unsigned-byte 8))
    (format t "~a~%" (read-byte in))
    (format t "~a~%" (read-byte in)))

201

NIL

101
201


# 路径名

In [26]:
; pathname对象: host, device, directory, name, type, version
; PATHNAME
; PATHNAME-DIRECTORY, PATHNAME-NAME, PATHNAME-TYPE

In [29]:
(pathname "/foo/bar/baz.txt")

#P"/foo/bar/baz.txt"

In [30]:
#p"/foo/bar/baz.txt"

#P"/foo/bar/baz.txt"

In [28]:
(let ((pn (pathname "/foo/bar/baz.txt")))
     (print (pathname-directory pn))
     (print (pathname-name pn))
     (print (pathname-type pn)))

"txt"


(:ABSOLUTE "foo" "bar") 
"baz" 
"txt" 

In [ ]:
; NAEMSTRING
; DIRECTORY-NAMESTRING
; FILE-NAMESTRING

In [33]:
(namestring #p"/foo/bar/baz.txt")
(directory-namestring #p"/foo/bar/baz.txt")
(file-namestring #p"/foo/bar/baz.txt")

"/foo/bar/baz.txt"

"/foo/bar/"

"baz.txt"

In [ ]:
; MAKE-PATHNAME
; MERGE-PATHNAMES
; ENOUGH-NAMESTRING

In [34]:
(make-pathname :directory '(:absolute "foo" "bar")
               :name "baz"
               :type "txt")

#P"/foo/bar/baz.txt"

In [37]:
(make-pathname :type "html" :defaults #p"/foo/bar/baz.txt")
(make-pathname :directory '(:relative "backups") :defaults #p"/foo/bar/baz.txt")

#P"/foo/bar/baz.html"

#P"backups/baz.txt"

In [38]:
(merge-pathnames #p"foo/bar.html" #p"/www/html/")
(merge-pathnames #p"foo/bar.html" #p"www/html/")

#P"/www/html/foo/bar.html"

#P"www/html/foo/bar.html"

In [39]:
(enough-namestring #p"/www/html/foo/bar.html" #p"/www/")

"html/foo/bar.html"

In [41]:
(merge-pathnames #p"foo.txt")

#P"/Users/zhoujiagen/Google Drive/workcache/jupyter-notebook/CommonLisp/foo.txt"

In [42]:
; 目录名
(make-pathname :directory '(:absolute "foo") :name "bar") ; file form
(make-pathname :directory '(:absolute "foo" "bar")) ; firectory form

#P"/foo/bar"

#P"/foo/bar/"

# 与文件系统交互

In [ ]:
; PROBE-FILE
; DIRECTORY
; DELETE-FILE, RENAME-FILE
; ENSURE-DIRECTORIES-EXIST

; FILE-WRITE-DATE, FILE-AUTHOR
; FILE-LENGTH, FILE-POSITION

In [47]:
(probe-file #p"data/name.txt")
(probe-file #p"data/name1.txt")

#P"/Users/zhoujiagen/Google Drive/workcache/jupyter-notebook/CommonLisp/data/name.txt"

NIL

In [53]:
(directory #p"../*")

(#P"/Users/zhoujiagen/Google Drive/workcache/jupyter-notebook/.DS_Store"
 #P"/Users/zhoujiagen/Google Drive/workcache/jupyter-notebook/.ipynb_checkpoints/"
 #P"/Users/zhoujiagen/Google Drive/workcache/jupyter-notebook/CommonLisp/"
 #P"/Users/zhoujiagen/Google Drive/workcache/jupyter-notebook/Ethereum/"
 #P"/Users/zhoujiagen/Google Drive/workcache/jupyter-notebook/SQL/"
 #P"/Users/zhoujiagen/Google Drive/workcache/jupyter-notebook/Shell/"
 #P"/Users/zhoujiagen/Google Drive/workcache/jupyter-notebook/python/"
 #P"/Users/zhoujiagen/Google Drive/workcache/jupyter-notebook/scala/")

In [54]:
(file-write-date #p"data/name.txt")
(file-author #p"data/name.txt")

3774650413

"zhoujiagen"

In [59]:
(with-open-file (in #p"data/name.txt")
    (print (file-length in))
    (print (file-position in))
    (read-char in)
    (print (file-position in)))

1


40 
0 
1 

# 其他IO类型

In [ ]:
; STRING-STREAM
; MAKE-STRING-INPUT-STREAM
; MAKE-STRING-OUTPUT-STREAM

; WITH-INPUT-FROM-STRING
; WITH-OUTPUT-TO-STRING

In [60]:
(let ((s (make-string-input-stream "1.23")))
     (unwind-protect (read s)
                     (close s)))

1.23

In [61]:
(with-input-from-string (s "1.23")
    (read s))

1.23

In [62]:
(with-output-to-string (out)
    (format out "hello, world ")
    (format out "~s" '(1 2 3)))

"hello, world (1 2 3)"

In [ ]:
; BROADCASE-STREAM, MAKE-BROADCAST-STREAM
; CONCATENATED-STREAM, MAKE-CONCATENATED-STREAM
; TWO-WAY-STREAM, MAKE-TWO-WAY-STREAM
; ECHO-STREAM, MAKE-ECHO-STREAM